# Create a cluster of custom shape from a bulk material

Create a cluster with shape provided by condition on coordinates and crystal orientation along the z-axis from a bulk material.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 


## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment


### 1.1. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("")

### 1.2. Set up cluster parameters

In [ ]:
RADIUS = 5.3  # in Angstroms
VACUUM = 10.0  # in Angstroms on each side
Z_ORIENTATION = (0, 0, 1)  # Miller indices of the slab orientation along the z-axis for the cluster
USE_CARTESIAN_COORDINATES = True
NAME = "Icosahedron"  # Name of the cluster

### 1.3. Set coordinates condition

In [ ]:
import numpy as np
from scipy.spatial import ConvexHull
from typing import List
from mat3ra.made.tools.helpers import CoordinateCondition


class CustomCoordinateCondition(CoordinateCondition):
    """Creates a regular polyhedron shape using SciPy's ConvexHull"""
    radius: float = 1
    center_coordinate: List[float] = [0.5, 0.5, 0.5]

    @property
    def _hull_planes(self):
        t = (1 + np.sqrt(5)) / 2
        vertices = np.array([
            [-1, t, 0], [1, t, 0], [-1, -t, 0], [1, -t, 0],
            [0, -1, t], [0, 1, t], [0, -1, -t], [0, 1, -t],
            [t, 0, -1], [t, 0, 1], [-t, 0, -1], [-t, 0, 1]
        ]) * self.radius / np.sqrt(1 + t*t)
        return ConvexHull(vertices).equations

    def condition(self, coordinate: List[float]) -> bool:
        """Returns True if point is inside the polyhedron"""
        point = np.array(coordinate) - self.center_coordinate
        return all(np.dot(plane[:3], point) + plane[3] <= 0
                  for plane in self._hull_planes)

    
condition = CustomCoordinateCondition(radius=RADIUS)

### 1.3. Get input material

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

## 2. Create the Target Material
### 2.1. Set the cluster configuration


In [ ]:
from mat3ra.made.tools.helpers import create_nanoparticle_from_material
material = materials[0]
cluster = create_nanoparticle_from_material(material=material,condition=condition, orientation_z=Z_ORIENTATION, vacuum_padding=VACUUM, use_cartesian_coordinates=USE_CARTESIAN_COORDINATES)

## 3. Visualize the Result(s)

In [ ]:
from utils.visualize import visualize_materials as visualize
visualize([{"material": cluster, "title": f"Cluster"},
           {"material": cluster, "title": f"Cluster", "rotation": "-90x"}])

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

cluster.name = f"{materials[0].name} {NAME}"
set_materials(cluster)